In [1]:
import pandas as pd
import numpy as np
import time

In [2]:
start_time = time.time()

In [3]:
df = pd.read_parquet("/content/rating.parquet.gzip")
df.head(5)

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [4]:
df = df.sort_values(by=["userId", "timestamp"])
df = df.reset_index(drop=True)
df["timestamp"] = pd.to_datetime(df.timestamp)

In [5]:
nb_column_name = "nb_previous_ratings"
feature_series = df.groupby("userId")["timestamp"].rank(method="first") - 1
df[nb_column_name] = feature_series

In [6]:
avg_column_name = "avg_ratings_previous"
rating_cumsum = df.groupby("userId").rating.cumsum()
avg_ratings_previous = rating_cumsum / (feature_series + 1)
avg_ratings_previous = avg_ratings_previous.shift(1)
avg_ratings_previous[df["nb_previous_ratings"] == 0.0] = np.nan
df[avg_column_name] = avg_ratings_previous
df.head(5)

,userId,movieId,rating,timestamp,nb_previous_ratings,avg_ratings_previous
0,1,924,3.5,2004-09-10 03:06:38,0.0,NaN
1,1,919,3.5,2004-09-10 03:07:01,1.0,3.5
2,1,2683,3.5,2004-09-10 03:07:30,2.0,3.5
3,1,1584,3.5,2004-09-10 03:07:36,3.0,3.5
4,1,1079,4.0,2004-09-10 03:07:45,4.0,3.5


In [7]:
df.to_parquet("rating_with_features.gzip", compression="gzip")

In [8]:
print(time.time()-start_time)

151.80994248390198
